In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pickle
import random
import numpy as np
import copy
import json

In [2]:
from Classification_Model import (MLPForClassification,
                                  train_model,
                                  eval_model,
                                  test_model)
from Helper_Functions import set_seed
from Dataset_Generation import (make_model_dataset,
                                make_model_dataset_AndOrAnd,
                                make_intervention_dataset_variable_intervention_all,
                                make_intervention_dataset_variable_intervention_first,
                                make_intervention_dataset_first_input_intervention,
                                make_intervention_dataset_AndOrAnd,
                                make_intervention_dataset_AndOr)
from RevNet import RevNet
from Rotation_Model import Rotation
from DAS import phi_class
from DAS_MLP import Distributed_Alignment_Search_MLP

In [3]:
DEVICE  = "cuda" #"cuda"/"cpu"
#Setting = "Both Equality Relations"
#Setting = "Left Equality Relation"
#Setting = "Identity of First Argument"
#Setting = "AndOrAnd"
Setting = "AndOr"


transformation_config = {"type"        : "Rotation",
                         "in_features" :         24}
"""
transformation_config = {"type"          : "RevNet",
                         "number_blocks" :       10,
                         "in_features"   :       16,
                         "hidden_size"   :      128}
"""
Max_Epochs                       = 50
Early_Stopping_Epochs            = 5
early_stopping_improve_threshold = 0.001

In [4]:
one_variable_settings = ["Identity of First Argument"]
two_variable_settings = ["Both Equality Relations","Left Equality Relation","AndOrAnd","AndOr"]
DAS_Original_tasks    = ["Both Equality Relations","Left Equality Relation","Identity of First Argument"]
AndOrAnd_tasks        = ["AndOrAnd","AndOr"]

In [5]:
#Helper Functions:

def chunk_list(input_list, batch_size):
    return [input_list[i:i + batch_size] for i in range(0, len(input_list), batch_size)]

# Removed as this is not possible for all task. 
# Further DAS compares its performance with the general performance so I decided to use a standard test set 
# rather than one which is made out of "intervened inputs"
"""
def make_test_intervened_input_DS(DS):
    DS_X=[]
    DS_y=[]
    for a_DS in DS:
        DS_y.append(a_DS["label"])
        DS_X.append(copy.deepcopy(a_DS["base"]))
        for i in range(2):
            if a_DS["intervention"][i]:
                DS_X[-1][8*i:8*(i+1)]=copy.deepcopy(a_DS["sources"][i])[8*i:8*(i+1)]
    return DS_X,DS_y
"""

'\ndef make_test_intervened_input_DS(DS):\n    DS_X=[]\n    DS_y=[]\n    for a_DS in DS:\n        DS_y.append(a_DS["label"])\n        DS_X.append(copy.deepcopy(a_DS["base"]))\n        for i in range(2):\n            if a_DS["intervention"][i]:\n                DS_X[-1][8*i:8*(i+1)]=copy.deepcopy(a_DS["sources"][i])[8*i:8*(i+1)]\n    return DS_X,DS_y\n'

In [6]:
#DAS Training Loop

results=[]
for acseed in [4287, 3837, 9097, 2635, 5137, 6442, 5234, 4641, 8039, 2266]:
    set_seed(acseed)
    model = MLPForClassification(input_size=transformation_config["in_features"])
    model.to(DEVICE)
    if Setting in DAS_Original_tasks:
        X_train,y_train = make_model_dataset(1048576,4,DEVICE)
        X_eval,y_eval   = make_model_dataset(10000,4,DEVICE)
        X_test,y_test   = make_model_dataset(10000,4,DEVICE)
    elif Setting in AndOrAnd_tasks:
        X_train,y_train = make_model_dataset_AndOrAnd(1048576,4,DEVICE)
        X_eval,y_eval   = make_model_dataset_AndOrAnd(10000,4,DEVICE)
        X_test,y_test   = make_model_dataset_AndOrAnd(10000,4,DEVICE)
        

    indexes=[None]
    indexes_num=list(range(X_train.shape[0]))
    random.shuffle(indexes_num)
    indexes+=chunk_list(indexes_num, 1048576//8)
    random.shuffle(indexes_num)
    indexes+=chunk_list(indexes_num, 1048576//8)
    """
    random.shuffle(indexes_num)
    indexes+=chunk_list(indexes_num, 1048576//4)
    random.shuffle(indexes_num)
    indexes+=chunk_list(indexes_num, 1048576//4)
    random.shuffle(indexes_num)
    indexes+=chunk_list(indexes_num, 1048576//4)#//20)#5 epochs each epoch split into 16 steps
    """
    
    Layers=[]
    Layers.append(("Layer1",model.mlp.h[0]))
    Layers.append(("Layer2",model.mlp.h[1]))
    Layers.append(("Layer3",model.mlp.h[2]))
    inter_dims=[]

    if Setting in two_variable_settings:
        inter_dims.append([list(range(0,transformation_config["in_features"]//2)),list(range(transformation_config["in_features"]//2,transformation_config["in_features"]))])
        inter_dims.append([list(range(0,2)),list(range(2,4))])
        inter_dims.append([list(range(0,1)),list(range(1,2))])
    elif Setting in one_variable_settings:
        inter_dims.append([list(range(0,transformation_config["in_features"]//2))])
        inter_dims.append([list(range(0,2))])
        inter_dims.append([list(range(0,1))])
    else:
        Exception("Unknown Setting")

    if Setting == "Both Equality Relations":
        DAS_Train = make_intervention_dataset_variable_intervention_all(1280000,4)
        DAS_Test  = make_intervention_dataset_variable_intervention_all(10000,4)
        DAS_Eval  = make_intervention_dataset_variable_intervention_all(10000,4)
    elif Setting == "Left Equality Relation":
        DAS_Train = make_intervention_dataset_variable_intervention_first(1280000,4)
        DAS_Test  = make_intervention_dataset_variable_intervention_first(10000,4)
        DAS_Eval  = make_intervention_dataset_variable_intervention_first(10000,4)
    elif Setting == "Identity of First Argument":
        DAS_Train = make_intervention_dataset_first_input_intervention(1280000,4)
        DAS_Test  = make_intervention_dataset_first_input_intervention(10000,4)
        DAS_Eval  = make_intervention_dataset_first_input_intervention(10000,4)
    elif Setting == "AndOrAnd":
        DAS_Train = make_intervention_dataset_AndOrAnd(1280000,4)
        DAS_Test  = make_intervention_dataset_AndOrAnd(10000,4)
        DAS_Eval  = make_intervention_dataset_AndOrAnd(10000,4)
    elif Setting == "AndOr":
        DAS_Train = make_intervention_dataset_AndOr(1280000,4)
        DAS_Test  = make_intervention_dataset_AndOr(10000,4)
        DAS_Eval  = make_intervention_dataset_AndOr(10000,4)
    else:
        Exception("Unknown Setting")

    # Not used anymore see explanation above:
    #X_test,y_test=make_test_intervened_input_DS(DAS_Test)
    #X_test=torch.stack(X_test).to(DEVICE)
    #y_test=torch.stack(y_test).to(DEVICE)
    results.append([])
    for ac_in in indexes:   
        results[-1].append({})
        model.train()
        for param in model.parameters():
            param.requires_grad = True  # This unfreezes the weights
        if ac_in is not None:# first DAS on untrained
            model=train_model(model,X_train[ac_in],y_train[ac_in],X_eval,y_eval,batch_size = 1024,epochs=1)
        accuracy=test_model(model,X_test,y_test)
        
        results[-1][-1]["accuracy"]=accuracy
        for LayerName,Layer in Layers:
            results[-1][-1][LayerName]={}
            for inter_dim in inter_dims:
                print(LayerName,":",inter_dim, flush=True)
        
                #Initialize transformation function
                if transformation_config["type"]=="Rotation":
                    p = Rotation(transformation_config["in_features"])
                elif transformation_config["type"]=="RevNet":
                    p = RevNet(number_blocks =  transformation_config["number_blocks"],
                               in_features   =  transformation_config["in_features"],
                               hidden_size   =  transformation_config["hidden_size"]
                              )
                else:
                    Exception("Unknown transformation function")
                p.to(DEVICE)
                p_inverse = p.inverse
                optimizer = optim.Adam(p.parameters(), lr=0.001)
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
                criterion = nn.CrossEntropyLoss()
                phi=phi_class(p,p_inverse,criterion,optimizer,scheduler)
        
                
        
                DAS_Experiment=Distributed_Alignment_Search_MLP(Model                = model,
                                                                Model_Layer          = Layer,
                                                                Train_Data_Raw       = DAS_Train,
                                                                Test_Data_Raw        = DAS_Test,
                                                                Eval_Data_Raw        = DAS_Eval,
                                                                Hidden_Layer_Size    = transformation_config["in_features"],
                                                                Variable_Dimensions  = inter_dim,
                                                                Transformation_Class = phi,
                                                                Device               = DEVICE)
        
                DAS_Experiment.train_test(batch_size=6400,
                                          epochs=Max_Epochs,
                                          mode=1,
                                          early_stopping_threshold=Early_Stopping_Epochs,
                                          early_stopping_improve_threshold=early_stopping_improve_threshold) #Train
        
                accuracy=DAS_Experiment.train_test(batch_size=6400,
                                                   mode=2)
                
                results[-1][-1][LayerName][str(inter_dim)]=accuracy
                DAS_Experiment.Cleanup()
                DAS_Experiment=None
                with open('results.json', 'w') as f:
                    json.dump(results, f)

/home/master/DAS/DAS_Experiment_V5.2_Cleanup/Dataset_Generation.py:360: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  return torch.tensor(model_inputs, dtype=torch.float32).to(device),torch.tensor(labels, dtype=torch.float32).to(device)


Layer1 : [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]
Epoch 1, Loss: 0.691378690302372 steps without improvement: 0 eval accuracy: 0.5786 best eval accuracy: 0.5786 learning rate: 0.001


KeyboardInterrupt: 